In [1]:
link = "https://www.reuters.com/resources/archive/us/{}{}{}.html"
reuters_db = '/home/ostapkharysh/Documents/bt_data/DB/Reuters'

In [2]:
from bs4 import BeautifulSoup as BS
import requests
from requests.exceptions import HTTPError
#import re
import datetime
#import pytz
from newspaper import Article
from db_manager import *
import sqlite3
from tqdm import tqdm


years = ['2013']#, '2014', '2015', '2016', '2017']
months = ['01',]# '02', '03', '04', '05', '06', '07', '08', '09', '10', '11', '12']
days = ['01', ]#'02', '03', '04', '05', '06', '07', '08', '09', '10', '11', '12', '13', '14', '15', '16', '17', '18', '19'
       #'20', '21', '22', '23', '24', '25', '26', '27', '28', '29', '30', '31']
    
    

In [ ]:
conn = create_connection(reuters_db)

for y in years:
    for m in months:
        for d in days:
            try:
                datetime.datetime(year=int(y),month=int(m),day=int(d))
            except ValueError:
                print("Day doesn't exist")
                continue
            try:
                r = requests.get(link.format(y, m, d))
                r.raise_for_status() # check if page found
            except HTTPError:
                print("No page found by this link!",r.status)
                continue
                    
            soup = BS(r.content, 'html.parser')
            span_list = soup.find_all('div' , attrs={'class':"headlineMed"})

            header = str(soup.find_all('title')).split()
            day, month, year = header[6], header[7], header[8]

            k = [[str(el).split('>')[1].split("=")[1].replace('"', ''), str(el).split()[-2]] for el in span_list
                 if not str(el).split('>')[1].split("=")[1].replace('"', '').startswith('http://www.reuters.com/news/video')]
            
            print(len(k))
            
            for link_time in k:
                try:
                    r = requests.get(link_time[0])
                    r.raise_for_status()
                    
                    article = Article(link_time[0].strip())
                    article.download()
                    article.parse()
                    title = article.title
                    text = article.text
                    time = link_time[1]
                    date = datetime.datetime.strptime("{} {} {} {}".format(month, day, year, time), '%b %d %Y %I:%M%p')

                    #print(str(date)+'\n'+title+'\n'+text+'\n'+link_time[0])
                    print(str(date)+'\n'+title+'\n'+link_time[0])
                 
                    with conn:
                        news = (str(date), title, text, link_time[0])
                        create_task(conn, news)
                except HTTPError:
                    print("No article found by this link!", article.url)
                    
                    continue
                print()
                
                
close_db(conn)

259
2013-01-01 23:52:00
Factbox: Key points in bill passed by Congress to avert "fiscal cliff"
http://www.reuters.com/article/2013/01/02/us-usa-fiscal-deal-idUSBRE90103820130102

2013-01-01 23:51:00
After fiscal win, Obama warns Congress on debt fight
http://www.reuters.com/article/2013/01/02/us-usa-fiscal-obama-idUSBRE8BT04S20130102

2013-01-01 23:51:00
Obama resumes Hawaii vacation after "fiscal cliff" bill passes
http://www.reuters.com/article/2013/01/02/us-usa-fiscal-obama-hawaii-idUSBRE90103220130102

2013-01-01 23:49:00
Obama resumes Hawaii vacation after 'fiscal cliff' bill passes
http://www.reuters.com/article/2013/01/02/usa-fiscal-obama-hawaii-idUSL1E9C207B20130102

2013-01-01 23:44:00
Indonesia Nov foreign tourist arrivals rise 5.94 pct y/y
http://www.reuters.com/article/2013/01/02/indonesia-economy-tourist-idUSJ9N09L00G20130102

2013-01-01 23:42:00
After fiscal win, Obama warns Congress on debt fight
http://www.reuters.com/article/2013/01/02/usa-fiscal-obama-idUSL1E9C2073201

In [110]:
r = requests.get(link.format("2013", "08", "09"))  # when does not load "2013", "08", "09"
r.status_code == 200
#259
#285
#=
#26

True

In [111]:
soup = BS(r.content, 'html.parser')
span_list = soup.find_all('div' , attrs={'class':"headlineMed"})  #, 

In [112]:
k = str(span_list[0]).split('>')[1].split("=")[1].replace('"', '')
k

'http://www.reuters.com/article/2013/08/10/ca-glancy-binkow-idUSnBw095771a+100+BSW20130810'

In [113]:
time = str(span_list[0]).split()[-2]
time

'11:32PM'

In [114]:
from newspaper import Article

In [119]:
article = Article(k.strip())

try:
    requests.get(k)
    print(requests.get(k))
    article = Article(k.strip())
except HTTPError:
    print('***FAILED TO DOWNLOAD***', article.url)

    
    
    """try:
    article.download()
except HTTPError:
    print('***FAILED TO DOWNLOAD***', article.url)"""

<Response [404]>


In [97]:
article.parse()

ArticleException: Article `download()` failed with 404 Client Error: Not Found for url: https://www.reuters.com/article/2013/08/10/ca-glancy-binkow-idUSnBw095771a+100+BSW20130810 on URL http://www.reuters.com/article/2013/08/10/ca-glancy-binkow-idUSnBw095771a+100+BSW20130810

In [10]:
article.authors

['Alec Macfarlane', 'Min Read']

In [11]:
print(datetime.datetime.utcnow())
print(datetime.datetime.now())
UTC_OFFSET = datetime.datetime.now().hour - datetime.datetime.utcnow().hour
UTC_OFFSET

2018-10-20 07:54:11.652337
2018-10-20 10:54:11.653018


3

In [12]:
timezone = pytz.timezone('Etc/Greenwich')


print(article.publish_date.tzinfo)
print(article.publish_date)
#print(timezone.localize(article.publish_date))

print(article.publish_date - datetime.timedelta(hours=UTC_OFFSET))
# A v reuters pyshe sho published at 11:59 PM 

tzutc()
2017-08-09 03:59:52+00:00
2017-08-09 00:59:52+00:00


In [13]:
article.text


"HONG KONG (Reuters Breakingviews) - McDonald’s is going large in China. It will have to ensure that breakneck growth does not bring indigestion.\n\nCustomers play using their mobile phones at a McDonald's store in Beijing, China January 9, 2017. REUTERS/Jason Lee\n\nOn Tuesday, the $126 billion burger giant said it planned to open 2,000 mainland stores by 2022. That will nearly double its footprint from 2,500 outlets now. That target is up one-third from a previous goal of 1,500 restaurants in five years, a goal that also originally included Hong Kong.\n\nThis underlines how the $2.1 billion sale of McDonald’s China business, which was agreed in January and closed last week, was not really a retreat. It now holds one-fifth of the business, which has the rights to run McDonald’s operations in China for 20 years, with the rest owned by Chinese conglomerate CITIC, CITIC’s private equity arm, and U.S. buyout firm Carlyle.\n\nExpanding makes sense. Bigger rival Yum China, which operates Pi

In [14]:
article.title

'McDonald’s boldly supersizes its China strategy'

In [28]:
title = str(soup.find_all('title')).split()
day = title[6]
month = title[7]
year = title[8]

In [38]:
print(time)
print(day)
print(month)
print(year)


11:59AM
09
Aug
2017


In [36]:
date = datetime.datetime.strptime("{} {} {} {}".format(month, day, year, time), '%b %d %Y %I:%M%p')

In [37]:
date

datetime.datetime(2017, 8, 9, 11, 59)

In [53]:
datetime.datetime(year=2016,month=2,day=30)

ValueError: day is out of range for month

In [9]:
r = requests.get('http://www.reuters.com/article/2013/08/10/ca-glancy-binkow-idUSnBw095771a+100+BSW20130810')

In [14]:
import sqlite3

In [15]:
from db_manager import *

In [17]:
reuters_db = '/home/ostapkharysh/Documents/bt_data/DB/Reuters'
conn = create_connection(reuters_db)
print(conn)
with conn:
    news = ('1', '12.12.12', "TITLE BY ME", "BLA BLA BLA BLA", "LINKY TO VASHNO")
    create_task(conn, news)   

OperationalError: database is locked

In [12]:
r = requests.get(link.format("2017", "08", "09"))  # when does not load "2013", "08", "09"
r.status_code == 200

soup = BS(r.content, 'html.parser')
span_list = soup.find_all('div' , attrs={'class':"headlineMed"})

In [16]:
k = [[str(el).split('>')[1].split("=")[1].replace('"', '') , str(el).split()[-2]] for el in span_list]

In [17]:
k

[['http://www.reuters.com/article/us-mcdonalds-china-breakingviews/mcdonalds-boldly-supersizes-its-china-strategy-idUSKBN1AP0AI',
  '11:59PM'],
 ['http://www.reuters.com/article/brief-bluefocus-communication-group-to-b/brief-bluefocus-communication-group-to-buy-28-pct-stake-in-data-tech-firm-for-14-mln-yuan-idUSL4N1KV1Y0',
  '11:58PM'],
 ['http://www.reuters.com/article/us-people-glen-campbell/countrys-rhinestone-cowboy-glen-campbell-dies-after-alzheimers-battle-idUSKBN1AO2DZ',
  '11:57PM'],
 ['http://www.reuters.com/article/brief-cosco-shipping-technology-plans-to/brief-cosco-shipping-technology-plans-to-buy-info-tech-and-network-related-assets-idUSH9N1KT003',
  '11:57PM'],
 ['http://www.reuters.com/article/northkorea-missiles-diplomacy/update-1-trump-appears-to-grant-china-banks-sanctions-reprieve-after-un-deal-idUSL1N1KV05L',
  '11:56PM'],
 ['http://www.reuters.com/article/brief-tangrenshen-group-sees-q1q3-fy-201/brief-tangrenshen-group-sees-q1q3-fy-2017-net-profit-to-up-50-pct-to-7

In [34]:
lst = ['http://www.reuters.com/article/2013/01/01/idUSnHUGb3hlqa+1c0+ONE20130101', '12:01AM'], ['http://www.reuters.com/article/2013/01/01/census-daily-feature-idUSnPnDC35367+160+PRN20130101', '12:01AM'], ['http://www.reuters.com/article/2013/01/01/idUSnMKW69664a+1c0+MKW20130101', '12:01AM'], ['http://www.reuters.com/news/video/videoStory?storyID', '11:35PM'], ['http://www.reuters.com/news/video/videoStory?storyID', '7:53PM'], ['http://www.reuters.com/news/video/videoStory?storyID', '7:31PM'], ['http://www.reuters.com/news/video/videoStory?storyID', '7:03PM'], ['http://www.reuters.com/news/video/videoStory?storyID', '7:00PM'], ['http://www.reuters.com/news/video/videoStory?storyID', '6:57PM'], ['http://www.reuters.com/news/video/videoStory?storyID', '4:49PM'], ['http://www.reuters.com/news/video/videoStory?storyID', '4:03PM'], ['http://www.reuters.com/news/video/videoStory?storyID', '2:03PM'], ['http://www.reuters.com/news/video/videoStory?storyID', '2:03PM'], ['http://www.reuters.com/news/video/videoStory?storyID', '12:37PM'], ['http://www.reuters.com/news/video/videoStory?storyID', '12:31PM'], ['http://www.reuters.com/news/video/videoStory?storyID', '12:05PM'], ['http://www.reuters.com/news/video/videoStory?storyID', '11:09AM']
r = [el for el in lst if not el[0].startswith('http://www.reuters.com/news/video')]

In [35]:
r

[['http://www.reuters.com/article/2013/01/01/idUSnHUGb3hlqa+1c0+ONE20130101',
  '12:01AM'],
 ['http://www.reuters.com/article/2013/01/01/census-daily-feature-idUSnPnDC35367+160+PRN20130101',
  '12:01AM'],
 ['http://www.reuters.com/article/2013/01/01/idUSnMKW69664a+1c0+MKW20130101',
  '12:01AM']]

hello
hello
hello
hello
hello
hello
hello
hello
hello
hello
hello
hello
hello
hello
hello
hello
hello
hello
hello
hello
